# Introduction to SALib: model example

Interactive examples for the clinic, Model Sensitivity Analysis with SALib at [CSDMS Annual Meeting 2019](https://csdms.colorado.edu/wiki/Form:Annualmeeting2019).

**Goal of this notebook:** Use Landlab to demonstrate the Sobol method implemented in SALib.

Clinic resources are stored in [this repository](https://github.com/nathanlyons/sensitivity_analysis_clinic_CSDMS_2019).

### Research question
Which earth surface process parameters most influence divide migration?

### Approach
* Run a model where divide migration is driven by base level fall.
* The landscape will evolve by stream power incision and linear diffusion:  
\begin{equation*}
\frac{\delta z}{\delta t} = U - KA^mS^n+D\Delta z
\end{equation*}
where $z$ is elevation [L], $t$ is time [T], $U$ is uplift [LT$^{-1}$], $K$ is often referred to as the  erodibility coefficient [T$^{-1}$L$^{1-2m}$], $A$ is drainage area [L$^{2}$], $S$ is channel slope [L/L], $m$ and $n$ are exponents, and $D$ is the diffusion coefficient [L$^2$T$^{-1}$]. 

* In model trials, vary process parameters within published intervals. Also vary the magnitude of base level fall.


| Factor           | Units           |Min             | Max             | Reference      |
|:---------------- | -----------------:|---------------:| ---------------:|:-------------- |
| uplift rate, *U* |m yr<sup>-1</sup>| 10<sup>-5</sup> | 10<sup>-3</sup> | Burbank et al. (1996) |
| erodibility, *K* |yr<sup>-1</sup>  | 10<sup>-7</sup> | 10<sup>-5</sup> | Stock and Montgomery (1999) |
| diffusivity, *D* |m<sup>2</sup> yr<sup>-1</sup>| 10<sup>-3</sup> | 10<sup>-1</sup> | Martin (2000) |
| base level fall  |m | 10<sup>-1</sup> | 10<sup>2</sup>| |

### Stages of the base level fall model

1. Evolve the landscape to steady state.
2. Drop base level.
3. Return to steady state.

### Model responses to analyze

* **Topographic relief at initial steady state** (at end of stage 1)
* **Time back to steady state** (time from onset of stage 2 to end of 3)
* **Divide migration distance** (from end of stage 1 to end of 3)

## 0. Update `numpy`. 

This is **necessary only if you are using Hydroshare during the clinic**. 

Run this cell only once - it takes a while.

In [ ]:
! conda update -y numpy
from IPython.display import display_html
display_html("<script>Jupyter.notebook.kernel.restart()</script>", raw=True)

## 1. Prepare the environment

In [1]:
from copy import deepcopy
from os import makedirs
from os.path import exists, join
from shutil import rmtree

from matplotlib.colors import LogNorm
from landlab import RasterModelGrid
from landlab.components import (FastscapeEroder, FlowAccumulator,
                                LinearDiffuser)
from landlab.io import read_esri_ascii, write_esri_ascii
from landlab.plot import channel_profile as prf, imshow_grid
from matplotlib import pyplot as plt
import numpy as np
from pandas import DataFrame, read_csv, set_option
from pprint import pprint
from SALib.analyze import sobol
from scipy.optimize import curve_fit

import experiment_funcs as ef
import model_funcs as mf
import plot_funcs as pf

In [ ]:
%matplotlib notebook

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

## 1. Load the factor levels

A file with the model input values was previously created. (The values in table are exponents to a base 10.)

In [ ]:
levels_df = read_csv('trial_factor_levels.txt', index_col=0)
set_option('display.width', 1000, 'display.max_rows', 10)
print(levels_df)

## 2. Run a model trial

### Load the values of the factors for a trial

In [ ]:
trial_id = 588

# Get trial parameters.
trial_index = np.where(levels_df.index == trial_id)[0][0]
trial_factors_df = levels_df.iloc[trial_index]
trial_factors = mf.calculate_factor_values(trial_factors_df.to_dict())

# Print factors.
pprint(trial_factors)

### Create a folder to store trial output

In [ ]:
trial_path = 'trial.{}'.format(trial_id)

# Delete directory if it exists.
if exists(trial_path):
    rmtree(trial_path)

makedirs(trial_path)

### Create a model grid

In [ ]:
nrows = 200
ncols = 100
dx = 100
mg = RasterModelGrid(nrows, ncols, dx)

# Create initial topography with random elevation values.
z = mg.add_zeros('node', 'topographic__elevation')
np.random.seed(1)
z += np.random.rand(z.size)

# Set boundary conditions.
mg.set_closed_boundaries_at_grid_edges(right_is_closed=True,
                                       top_is_closed=False,
                                       left_is_closed=True,
                                       bottom_is_closed=False)

# Plot grid elevation.
imshow_grid(mg, 'topographic__elevation', cmap='gray',
            colorbar_label='elevation (m)')

### Run the model

In [ ]:
# Set time step duration.
dt = 1000

f = trial_factors

# Instantiate model components.
fa = FlowAccumulator(mg, flow_director='D8')
sp = FastscapeEroder(mg, K_sp=f['K'], m_sp=0.5, n_sp=1)
ld = LinearDiffuser(mg, linear_diffusivity=f['D'], deposit=False)

# Instantiate model components.
fa = FlowAccumulator(mg, flow_director='D8')
sp = FastscapeEroder(mg, K_sp=f['K'], m_sp=0.5, n_sp=1)
ld = LinearDiffuser(mg, linear_diffusivity=f['D'], deposit=False)

# Set variables to evaluate presence of steady state.
initial_conditions_set = False
at_steady_state = False
relief_record = []
recent_mean = []
recent_std = []
step = 0

# Set number of time steps, `steps_ss` that is the time window to evaluate
# steady state.
steps_ss = 1000

# Create a dictionary to store responses.
response = {}

# Run model until steady state is reached.

uplift_per_step = f['U'] * dt
core_mask = mg.node_is_core()

print('Running model until elevation reaches steady state.')

while not at_steady_state:
    # Step processes.
    fa.run_one_step()
    sp.run_one_step(dt)
    ld.run_one_step(dt)
    
    # Uplift topography.
    z[core_mask] += uplift_per_step

    # Run until mean and std change of z is < 1% over past 1000 steps.
    at_steady_state = mf.check_steady_state(step * dt, z, step,
                                            steps_ss, relief_record,
                                            recent_mean, recent_std)

    # Advance model stage when steady state is reached.
    if at_steady_state and not initial_conditions_set:
        # Begin stage 2.
        initial_conditions_set = True
        time_to_initial_steady_state = step * dt

        # Save elevation of the initial conditions.
        fn = join(trial_path, 'initial_elevation.asc')
        write_esri_ascii(fn, mg, ['topographic__elevation'], clobber=True)

        # Retain steady state relief, `relief_ss`.
        z_core = z[mg.core_nodes]
        relief_ss = z_core.max() - z_core.min()
        response['relief_at_steady_state'] = relief_ss

        # Find steady state divide position.
        divide_y_coord_initial = mf.get_divide_position(mg)

        # Perturb elevation.
        base_level_nodes = mg.y_of_node == 0
        z[base_level_nodes] -= f['base_level_fall']
        at_steady_state = False

    elif at_steady_state and initial_conditions_set:
        # Reached stage 3.
        print('Time back to steady state:', step * dt)
        response['time_back_to_steady_state'] = step * dt

        # Get divide migration distance.
        divide_y_coord_final = mf.get_divide_position(mg)
        d = divide_y_coord_final - divide_y_coord_initial
        response['divide_migration_distance'] = d

        # Save final elevation.
        fn = join(trial_path, 'final_elevation.asc')
        write_esri_ascii(fn, mg, ['topographic__elevation'], clobber=True)

    # Advance step counter.
    step += 1

# Write response to file.
path = join(trial_path, 'response.csv')
ef.write_data(response, path)

### Compare initial and final grids

In [ ]:
# Calculate final relief.
z_core = z[mg.core_nodes]
relief_final = z_core.max() - z_core.min()

# Create summary DataFrame.
titles = ['initial steady state', 'final steady state']
DataFrame({'grid': titles,
           'time to steady state (yr)': [time_to_initial_steady_state, response['time_back_to_steady_state']],
           'relief (m)': [relief_ss, relief_final],
           'divide y-coordinate (m)': [divide_y_coord_initial, divide_y_coord_final]})

### Plot steady state grids and elevation profiles

In [ ]:
fig1, axes1 = plt.subplots(1, 2, figsize=(7, 4))
fig2, axes2 = plt.subplots(1, 1)

file_names = ['initial_elevation.asc', 'final_elevation.asc']

for i, fn in enumerate(file_names):
    # Plot grid.
    path = join(trial_path, fn)
    mgi, zi = read_esri_ascii(path, name='topographic__elevation')
    plt.sca(axes1[i])
    imshow_grid(mgi, 'topographic__elevation', cmap='gray',
                colorbar_label='elevation (m)')
    axes1[i].set_title(titles[i])
    
    # Plot main divide on grid.
    zi = mgi.at_node['topographic__elevation'].reshape(mgi.shape)
    z_row_mean = zi.mean(axis=1)
    divide_y_coord = z_row_mean.argmax() * mgi.dy
    axes1[i].plot([mgi.node_x.min(), mgi.node_x.max()],
                  [divide_y_coord, divide_y_coord], '--', color='r')
    
    # Plot elevation profile.
    axes2.plot(np.unique(mgi.node_y), z_row_mean, ['darkgray', 'k'][i],
              label=titles[i])
    axes2.set_xlabel('y coordinate (m)')
    axes2.set_ylabel('mean elevation (m)')

plt.figure(fig1.number)
plt.tight_layout()
    
plt.figure(fig2.number)
l = plt.legend()

## 3. Sensitivity analysis

Above, we looked at a single model trial. Now we turn to analzing the results of many trials.

Let's set up the problem dictionary following the approach at the top of this notebook. Note that the bounds of the variable, `base_level_fall` is also an exponent.

In [ ]:
problem = {
    'num_vars': 4,
    'names': ['U_exp', 'K_exp', 'D_exp', 'base_level_fall'],
    'bounds': [[-5, -3],
               [-7, -5],
               [-3, -1],
               [-1,  2]]}

pprint(problem)

Model trials were previously completed. Load the trial responses.

In [ ]:
summary_df = read_csv('data_summary.csv', index_col=0)
print(summary_df)

### Call `SALib.analyze.sobol.analyze` for each response.

In [ ]:
# divide_migration_distance  relief_at_steady_state  time_back_to_steady_state
response = 'divide_migration_distance'

Y = summary_df[response].values
Si = sobol.analyze(problem, Y)

pf.plot_sobol_indices(Si, problem, title=response, factor_names=['U', 'K', 'D', 'base level fall'])

## 4. Plot data

Compare model input parameters to responses in the plots created below.

In [ ]:
# Load data.
U = 10**summary_df.U_exp
K = 10**summary_df.K_exp
D = 10**summary_df.D_exp
blf = 10**summary_df.base_level_fall
R = summary_df.relief_at_steady_state
dmd = summary_df.divide_migration_distance
tss = summary_df.time_back_to_steady_state

# Plot.
x = U
y = R
color = K

fig, ax = plt.subplots(nrows=1, ncols=1)
ax.set_xscale('log')
ax.set_yscale('log')
s = ax.scatter(x, y, c=color, s=3, cmap='plasma',
               norm=LogNorm(vmin=color.min(), vmax=color.max()))

ax.set_xlabel('$U$ (m yr$^{-1}$)')
ax.set_ylabel('topographic relief (m)')

cb = plt.colorbar(s)
cb.set_label('$K$ (yr$^{-1}$)')

In [ ]:
x = blf / R
y = dmd
color = K

fig, ax = plt.subplots(nrows=1, ncols=1)
ax.set_xscale('log')

s = ax.scatter(x, y, c=color, s=3, cmap='plasma',
               norm=LogNorm(vmin=color.min(), vmax=color.max()))

ax.set_xlabel('base level fall (m) / relief (m)')
ax.set_ylabel('divide migration distance (m)')

cb = plt.colorbar(s)
cb.set_label('$K$ (yr$^{-1}$)')

In [ ]:
x = U / K
y = tss * 0.0001
color = blf / R

fig, ax = plt.subplots(nrows=1, ncols=1)
ax.set_xscale('log')
s = ax.scatter(x, y, c=color, s=3, cmap='plasma',
               norm=LogNorm(vmin=color.min(), vmax=color.max()))

ax.set_xlabel('U (m yr$^{-1}$) / K (yr$^{-1}$)')
ax.set_ylabel('time back to steady state (kyr)')

cb = plt.colorbar(s)
cb.set_label('base level fall (m) / relief (m)')